In [1]:
# 選手情報・過去レース情報から3連単舟券120種をクラス分類する
# こちらではパラメタのベイズ最適化を試みる。
# todo データの項目を増やしてみる。スタートタイムや過去の連対率。逆に、オッズは消す。

# 汎用ライブラリのimport
import sys
import os
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math
import tensorflow as tf
import collections

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from scipy.stats import rankdata
from sklearn import metrics
import warnings

In [2]:
# 自作ライブラリのimport
if os.environ['BR_HOME']+"/boatrace" not in sys.path:
    sys.path.append(os.environ['BR_HOME']+"/boatrace")
#print(sys.path)

from setup.myUtil import dbHandler


In [3]:
# 分析期間の指定は一旦ここでまとめてみる。
trainStartDate="20170101"
trainEndDate="20181231"
# test はtrainからsplitする

In [4]:
dbh=dbHandler.getDBHandle()
#dbHandler.closeDBHandle(dbh)

In [5]:
# trainの元データを取得
with dbh.cursor() as cursor:
    sel_sql = "select * from raceabst_forml_v \
               where raceDate between '%s' and '%s' \
               order by raceId "\
               % (trainStartDate,trainEndDate)
    cursor.execute(sel_sql)
    loadList=cursor.fetchall()
print("traindata:",len(loadList))

traindata: 94959


In [6]:
df = pd.io.json.json_normalize(loadList)
df.head()

,funaken,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1rank,l1score,l2Fcnt,l2boat2r,...,l6boat3r,l6motor2r,l6motor3r,l6rank,l6score,odds,raceDate,raceId,raceWaveHeight,raceWindSpeed
0,1-5-2,0,0.3333,0.4848,0.3817,0.5645,B1,4.842381,0,0.3333,...,0.4452,0.3134,0.4478,B2,0.594831,30.3,2017-04-01,20170401-02-01,1,2.0
1,2-1-3,1,0.2876,0.4248,0.3960,0.6040,B1,7.406036,0,0.3871,...,0.4177,0.4500,0.5750,B1,1.504087,79.0,2017-04-01,20170401-02-02,1,2.0
2,2-3-1,1,0.2452,0.4387,0.4610,0.5779,B1,8.340631,1,0.3807,...,0.4938,0.3541,0.5502,A2,3.195143,28.9,2017-04-01,20170401-02-03,1,3.0
3,1-4-2,0,0.1959,0.3986,0.6369,0.7738,B1,9.654840,0,0.3439,...,0.3846,0.3161,0.4540,B1,0.583962,7.7,2017-04-01,20170401-02-04,1,3.0
4,1-2-4,0,0.3609,0.4852,0.2983,0.4475,A2,15.248551,0,0.2911,...,0.5375,0.4296,0.6667,A2,2.543009,52.7,2017-04-01,20170401-02-05,1,2.0


In [7]:
# 入力のデータ整形
xdf=df.drop(['funaken','odds','raceId','raceDate'],axis=1)
#xdf=pd.get_dummies(xdf,columns=['l1rank','l2rank','l3rank','l4rank','l5rank','l6rank'])
rankLabel=LabelEncoder()
rankLabel=rankLabel.fit(xdf['l1rank'])
xdf['l1rank']=rankLabel.transform(xdf['l1rank'])
xdf['l2rank']=rankLabel.transform(xdf['l2rank'])
xdf['l3rank']=rankLabel.transform(xdf['l3rank'])
xdf['l4rank']=rankLabel.transform(xdf['l4rank'])
xdf['l5rank']=rankLabel.transform(xdf['l5rank'])
xdf['l6rank']=rankLabel.transform(xdf['l6rank'])
xdf.head()


,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1rank,l1score,l2Fcnt,l2boat2r,l2boat3r,...,l5score,l6Fcnt,l6boat2r,l6boat3r,l6motor2r,l6motor3r,l6rank,l6score,raceWaveHeight,raceWindSpeed
0,0,0.3333,0.4848,0.3817,0.5645,2,4.842381,0,0.3333,0.5094,...,2.860248,0,0.2740,0.4452,0.3134,0.4478,3,0.594831,1,2.0
1,1,0.2876,0.4248,0.3960,0.6040,2,7.406036,0,0.3871,0.5935,...,2.015905,1,0.2532,0.4177,0.4500,0.5750,2,1.504087,1,2.0
2,1,0.2452,0.4387,0.4610,0.5779,2,8.340631,1,0.3807,0.5739,...,1.279918,0,0.3125,0.4938,0.3541,0.5502,1,3.195143,1,3.0
3,0,0.1959,0.3986,0.6369,0.7738,2,9.654840,0,0.3439,0.5350,...,1.545834,0,0.2231,0.3846,0.3161,0.4540,2,0.583962,1,3.0
4,0,0.3609,0.4852,0.2983,0.4475,1,15.248551,0,0.2911,0.4557,...,11.021105,1,0.3500,0.5375,0.4296,0.6667,1,2.543009,1,2.0


In [8]:
# 結果のOne-Hot表現を作る⇒LGBMは数値配列なので数字にする。
ydf=df['funaken']
yLabel = LabelEncoder()
yLabel = yLabel.fit(ydf)
ydf = pd.DataFrame(yLabel.transform(ydf))
#ydf = yLabel.transform(ydf)
#ydf=pd.get_dummies(ydf,columns=['funaken'])
ydf.head()
#ydf.describe()

,0
0,12
1,20
2,24
3,8
4,1


In [9]:
# 重み付けのため、オッズのリストを作る
#odf=df['odds'].values
odf=pd.DataFrame(df['odds'])
#odf.describe()
print(type(odf))

<class 'pandas.core.frame.DataFrame'>


In [10]:
bayesian_tr_index, bayesian_val_index  = list(StratifiedKFold(n_splits=2, shuffle=True, random_state=1).split(xdf, ydf))[0]


In [11]:
def LGB_bayesian(
    num_leaves, #int
    min_data_in_leaf, #int
    reg_alpha,
    reg_lambda,
    max_depth #int
):
    
    # 整数じゃないといけないパラメータ。
    num_leaves = int(num_leaves)
    min_data_in_leaf = int(min_data_in_leaf)
    max_depth = int(max_depth)    
    assert type(num_leaves)==int
    assert type(min_data_in_leaf)==int
    assert type(max_depth)==int
    
    params={
        # 多値分類問題
        'objective': 'multiclass',
        # クラス数は 120
        'num_class': 120,
        #'class_weight':'balanced',
        'random_state':999,
        # 以下、ハイパーパラメタ
        'max_depth':max_depth,
        'num_leaves':num_leaves,
        'min_data_in_leaf':min_data_in_leaf,
        # 正則化
        'reg_alpha':reg_alpha,
        'reg_lambda':reg_lambda,
    }

    xg_train = lgb.Dataset(xdf.iloc[bayesian_tr_index],ydf.iloc[bayesian_tr_index])
    xg_valid = lgb.Dataset(xdf.iloc[bayesian_val_index],ydf.iloc[bayesian_val_index])

    evals_result = {}
    num_round = 5000
    clf = lgb.train(params, xg_train, num_round, valid_sets = [xg_valid], verbose_eval = 250 ,early_stopping_rounds = 50,evals_result=evals_result)
    #print(evals_result['eval']['multi_logloss'])
    #print(evals_result['valid_0']['multi_logloss'])
    print(min(evals_result['valid_0']['multi_logloss']))

    predictions = clf.predict(xdf.iloc[bayesian_val_index], num_iteration=clf.best_iteration)   
    
    #score = metrics.roc_auc_score(xdf.iloc[bayesian_val_index],predictions)
    # 精度 (Accuracy) を計算する
    #print(predictions)
    score=1/min(evals_result['valid_0']['multi_logloss'])
 
    return score

In [12]:
bounds_LGB={
    'max_depth':(3,15),
    'min_data_in_leaf':(0,300),
    'num_leaves':(5,20),
    'reg_alpha':(0,10.0),
    'reg_lambda':(0,10.0)
}

In [13]:
LGB_BO = BayesianOptimization(LGB_bayesian, bounds_LGB, random_state=13)
print(LGB_BO.space.keys)

['max_depth', 'min_data_in_leaf', 'num_leaves', 'reg_alpha', 'reg_lambda']


In [14]:
init_points = 5
n_iter = 10

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=init_points, n_iter=n_iter, acq='ucb', xi=0.0, alpha=1e-6)

|   iter    |  target   | max_depth | min_da... | num_le... | reg_alpha | reg_la... |
-------------------------------------------------------------------------------------
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[88]	valid_0's multi_logloss: 3.90548
3.905477907432176
|  1        |  0.2561   |  12.33    |  71.26    |  17.36    |  9.657    |  9.726    |
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[79]	valid_0's multi_logloss: 3.90279
3.9027891094577147
|  2        |  0.2562   |  8.441    |  182.7    |  16.63    |  6.416    |  7.22     |
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[148]	valid_0's multi_logloss: 3.87956
3.879559787564874
|  3        |  0.2578   |  3.42     |  89.53    |  5.878    |  8.571    |  3.729    |
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[72]	valid_0